In [ ]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


#1. Scrape Web Pages

1. BBC Weather

In [ ]:
#import packages
import os

import requests               # to get the webpage
import json                   # to convert API to json format
from bs4 import BeautifulSoup

from urllib.parse import urlencode
import numpy as np
import pandas as pd
import re                     # regular expression operators

from datetime import datetime

In [ ]:
# Creating a function to output location id by taking any city name as input.

def getlocid(city):
    city = city.lower() # convert city name to lowercase to standardize format
    # Convert into an API call using URL encoding
    location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
      'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
      's': city,
      'stack': 'aws',
      'locale': 'en',
      'filter': 'international',
      'place-types': 'settlement,airport,district',
      'order': 'importance',
      'a': 'true',
      'format': 'json'
    })
    result = requests.get(location_url).json()
    locid = result['response']['results']['results'][0]['id']
    return locid

In [ ]:
#We now GET the webpage of interest, from the server

required_city = 'Mumbai'
locid = getlocid(required_city)
url      = 'https://www.bbc.com/weather/' + str(locid)
response = requests.get(url)

In [ ]:
#Next, we initiate an instance of BeautifulSoup

soup = BeautifulSoup(response.content,'html.parser')
for i in soup.find_all('span', attrs='class'):
  print(i)

In [ ]:
'''
The information we want (daily high and low temp., and daily weather
summary), are in specific blocks on the webpage. We need to find the block type,
type of identifier, and the identifier name (all these can be figured out by
right clicking on the webpage and selecting 'Inspect' on the Chrome browser;
similar modus operandi for other browsers)
'''

#Temperature predictions
daily_high_values = soup.find_all('span', attrs={'class': 'wr-day-temperature__high-value'}) # block-type: span; identifier type: class; and class name: wr-day-temperature__high-value
daily_low_values  = soup.find_all('span', attrs={'class': 'wr-day-temperature__low-value'})

#Weather predictions
daily_summary = soup.find('div', attrs={'class': 'wr-day-summary'})

In [ ]:
'''
With the code snippet in the cell above, we get forecast data for 14 days,
including today. We will now post process the data to first extract the required
information/text and discard all the html wrapper code, then combine all
variables into one common list, and finally convert it into a pandas data frame
'''

daily_high_values_list = [daily_high_values[i].text.strip().split()[0] for i in range(len(daily_high_values))]
daily_low_values_list = [daily_low_values[i].text.strip().split()[0] for i in range(len(daily_low_values))]
daily_summary_list = re.findall('[a-zA-Z][^A-Z]*', daily_summary.text) #split the string on uppercase
datelist = pd.date_range(datetime.today(), periods=len(daily_high_values)).tolist()
datelist = [datelist[i].date().strftime('%y-%m-%d') for i in range(len(datelist))]
zipped = zip(datelist, daily_high_values_list, daily_low_values_list, daily_summary_list)
df = pd.DataFrame(list(zipped), columns=['Date', 'High','Low', 'Summary'])
display(df)

,Date,High,Low,Summary
0,22-06-25,30°,24°,Light rain showers and a gentle breeze
1,22-06-26,30°,26°,Light cloud and a gentle breeze
2,22-06-27,30°,26°,Thundery showers and a moderate breeze
3,22-06-28,30°,26°,Thundery showers and a gentle breeze
4,22-06-29,29°,26°,Thundery showers and a gentle breeze
5,22-06-30,28°,26°,Thundery showers and a moderate breeze
6,22-07-01,29°,26°,Thundery showers and a moderate breeze
7,22-07-02,30°,26°,Thundery showers and a moderate breeze
8,22-07-03,29°,26°,Thundery showers and a moderate breeze
9,22-07-04,29°,26°,Thundery showers and a moderate breeze


In [ ]:
# remove the 'degree' character
df.High = df.High.replace('\°','',regex=True).astype(float)
df.Low  = df.Low.replace('\°','',regex=True).astype(float)
display(df)

,Date,High,Low,Summary
0,22-06-25,30.0,24.0,Light rain showers and a gentle breeze
1,22-06-26,30.0,26.0,Light cloud and a gentle breeze
2,22-06-27,30.0,26.0,Thundery showers and a moderate breeze
3,22-06-28,30.0,26.0,Thundery showers and a gentle breeze
4,22-06-29,29.0,26.0,Thundery showers and a gentle breeze
5,22-06-30,28.0,26.0,Thundery showers and a moderate breeze
6,22-07-01,29.0,26.0,Thundery showers and a moderate breeze
7,22-07-02,30.0,26.0,Thundery showers and a moderate breeze
8,22-07-03,29.0,26.0,Thundery showers and a moderate breeze
9,22-07-04,29.0,26.0,Thundery showers and a moderate breeze


In [ ]:
# create a recording
filename_csv = required_city +'.csv'
df.to_csv(filename_csv, index=None)



---



---



2. Scraping off IMDb

In [ ]:
#import packages
from bs4 import BeautifulSoup as bs
import requests #to access website
import pandas as pd

In [ ]:
r = requests.get("https://www.imdb.com/chart/top/")

# Convert to a beautiful soup object
soup = bs(r.content)

# Print out HTML
contents = soup.prettify()

In [ ]:
#Creating empty lists

movie_title = []
movie_year = []
movie_rating = []


In [ ]:
#Extracting contents of page

imdb_table = soup.find(class_="chart full-width")
movie_titlecolumn = imdb_table.find_all(class_="titleColumn")
movie_ratingscolumn = imdb_table.find_all(class_="ratingColumn imdbRating")

for row in movie_titlecolumn:
    title = row.a.text # tag content extraction
    movie_title.append(title)

for row in movie_titlecolumn:
    year = row.span.text # tag content extraction
    movie_year.append(year)

for row in movie_ratingscolumn:
    rating = row.strong.text # tag content extraction
    movie_rating.append(rating)

In [ ]:
#creating DataFrame
movie_df = pd.DataFrame({'Movie Title': movie_title, 'Year of Release': movie_year, 'IMDb Rating': movie_rating})
movie_df.head()

,Movie Title,Year of Release,IMDb Rating
0,The Shawshank Redemption,(1994),9.2
1,The Godfather,(1972),9.2
2,The Dark Knight,(2008),9.0
3,The Godfather Part II,(1974),9.0
4,12 Angry Men,(1957),8.9




---



---



3. Scraping using Excel
\
Data -> New Query ->From Other Sources -> From Web

#2. Extracting from Wikipedia

In [ ]:
#import packages
!pip install wikipedia
import wikipedia as wk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=231a1971b9de3e0789c6f8a76224b0abf1787325d1b6fff41e65ef363c336365
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [ ]:
#search giving all results
wk.search("IIT Madras")

#search giving top 5 results
wk.search("IIT Madras", results=5)

#summary of search term
wk.summary('IIT Madras')

#storing the page as an object
full_page = wk.page('IIT Madras')

#can call methods on this page objec
full_page.summary
full_page.content
full_page.url
full_page.references
full_page.title
full_page.images
full_page.html()

In [ ]:
#extract html code of wikipedia page based on any search text
html = wk.page("IIT Madras").html().encode("UTF-8")

import pandas as pd
df = pd.read_html(html)[3]
df

University and college rankings                            
        General â international General â international.1
0                 ARWU (2021)[28]                     701-800
1           QS (World) (2022)[29]                         255
2            QS (Asia) (2021)[30]                          50
3           QS (BRICS) (2019)[31]                          17
4               General â India           General â India
5       NIRF (Overall) (2021)[32]                           1
6           Engineering â India       Engineering â India
7                 NIRF (2021)[33]                           1



---



---



#3. Geocoding

In [ ]:
#import packages
from geopy.geocoders import Nominatim

In [ ]:
#activate nominatim geocoder
locator = Nominatim(user_agent="21f1005301")

#type any address text
location = locator.geocode("Champ de Mars, Paris, France")

#print lattitude and longitude of the address
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

#the API output has multiple other details as a json like altitude, lattitude, longitude, correct raw addres, etc
location.raw
location.point
location.longitude
location.latitude
location.altitude
location.address

Latitude = 48.85614465, Longitude = 2.297820393322227


'Champ de Mars, Rue Edgar Faure, Quartier de Grenelle, Paris 15e Arrondissement, Paris, Île-de-France, France métropolitaine, 75015, France'

In [ ]:
#obtaining distance between any 2 cities:
def lat_long(city):
  location = locator.geocode(city)
  return location.latitude, location.longitude

def dist(city1,city2):
  c1, c2 = lat_long(city1), lat_long(city2)
  from geopy.distance import geodesic
  return geodesic(c1,c2).km

dist("Mumbai","Chennai")

cities = ['Nalgonda', 'Gangavati', 'Nagpur', 'Jaipur']
distance = {}
for city in cities:
  distance[city] = dist(city,'Chandrapur')
print(distance)

print(dist('Thiruvananthapuram','Mandsaur'))
print(dist('Nizamabad','Chhindwara'))
print(dist('Mandya','Vizianagaram'))
print(dist('Machilipatnam','Mangaluru'))

{'Nalgonda': 359.768605722938, 'Gangavati': 594.3310519398798, 'Nagpur': 124.62528085669706, 'Jaipur': 843.6436738989537}
1749.8150688827466
607.3545583710362
932.9792714733229
770.7109670427649




---



---



#4. PDF Scraping

In [ ]:
import os
import pandas as pd
import requests
import urllib.request
import pandas as pd
from urllib.parse import urljoin
from bs4 import BeautifulSoup

# Tabula scrapes tables from PDFs
!pip install tabula-py
import tabula

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
filee = '/content/custom.pdf'
table = tabula.read_pdf(filee,pages='all',multiple_tables=False)
table[0].to_csv('/content/custom.csv')

In [ ]:
#scraping off the first page
#data1 is at https://nbviewer.org/github/kuruvasatya/Scraping-Tables-from-PDF/blob/master/data1.pdf
file1 = '/content/data1.pdf'
table1 = tabula.read_pdf(file1)
table1[0]

'pages' argument isn't specified.Will extract only from page 1 by default.


,Name,Maths,Physics,Chemistry,Total
0,A,76,98,68,242
1,B,92,82,86,260
2,C,53,92,90,235
3,D,64,87,73,224
4,E,52,68,52,172
5,F,51,52,68,171
6,G,75,53,50,178
7,H,62,67,64,193
8,I,64,85,71,220
9,J,52,82,91,225


In [ ]:
#scraping off any page
#data2 is at https://nbviewer.org/github/kuruvasatya/Reading-Table-Data-From-PDF/blob/master/data.pdf
file2 = '/content/data2.pdf'
table2 = tabula.read_pdf(file2,pages='2')
table2

[   Name  Score
 0     A     40
 1     B     26
 2     C     22
 3     D      2
 4     E     51
 5     F    100
 6     G     55
 7     H     91
 8     I     53
 9     J     24
 10    K     95
 11    L     66
 12    M     45
 13    N     42
 14    O     48
 15    P     92
 16    Q     74
 17    R     38
 18    S     73
 19    T    100
 20    U     10
 21    V     94
 22    W     63
 23    X     26
 24    Y     58]

In [ ]:
#scraping multiple tables off a single page
#data3 is at https://nbviewer.org/github/kuruvasatya/Reading-Table-Data-From-PDF/blob/master/data3.pdf
file3 = '/content/data3.pdf'

#reading the tables are individual tables
tables = tabula.read_pdf(file3 ,pages=1, multiple_tables=True)
print(tables[0])
print(tables[1])

#reading the tables are one table
tables = tabula.read_pdf(file3 ,pages=1,multiple_tables=False)
print(tables[0])

  Name  Score
0    A     91
1    B     14
2    C     15
3    D     66
4    E     16
  Name University
0    A      Texas
1    B      Texas
2    C      Texas
3    D      Texas
4    E      Texas
    Name       Score
0      A          91
1      B          14
2      C          15
3      D          66
4      E          16
5   Name  University
6      A       Texas
7      B       Texas
8      C       Texas
9      D       Texas
10     E       Texas


In [ ]:
# output just the first page tables in the PDF to a CSV
tabula.convert_into('data2.pdf', "file2.csv", pages=1)

#output all tables in the PDF to a CSV
tabula.convert_into('data3.pdf', "file3.csv", pages = 'all')

In [ ]:
# Save contents from url into folder_location
url = 'https://www.premierleague.com/publications'
folder_location = '/content/premier_league'
if not os.path.exists(folder_location):
    os.mkdir(folder_location)

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Loop through all PDF links in the page
for link in soup.select("a[href$='.pdf']"):
    # Local file name is the same as PDF file name in the URL (ignoring rest of the path)
    # https://premierleague-static-files.s3.amazonaws.com/premierleague/document/2016/07/02/e1648e96-4eeb-456e-8ce0-d937d2bc7649/2011-12-premier-league-season-review.pdf
    filename = os.path.join(folder_location, link['href'].split('/')[-1])
    with open(filename, 'wb') as f:
        f.write(requests.get(urljoin(url,link['href'])).content)


In [ ]:
#parsing a table in one of the files
combined_pdf = folder_location + '/2011-12-premier-league-season-review.pdf'
tabula.read_pdf(combined_pdf,  pages='59')

[   THe PReMieR LeaGUe TOTaL BROadcaSTinG PaYMenTS SeaSOn 2011/12  \
 0                 as at 14 May 2012 (all figues in £)              
 1                   Place Live BBC n.Live Equal Share              
 2                       arsenal 3 19 38 19 13,788,093              
 3                  aston Villa 16 10 38 28 13,788,093              
 4             Blackburn Rovers 19 11 38 27 13,788,093              
 5             Bolton Wanderers 18 10 38 28 13,788,093              
 6                       chelsea 6 20 38 18 13,788,093              
 7                       Everton 7 10 38 28 13,788,093              
 8                        Fulham 9 10 38 28 13,788,093              
 9                     Liverpool 8 23 38 15 13,788,093              
 10              Manchester city 1 25 38 13 13,788,093              
 11            Manchester united 2 26 38 12 13,788,093              
 12             newcastle united 5 18 38 20 13,788,093              
 13                norwich city 12

In [ ]:
#reading from a particular area of the page
combined_pdf = folder_location + "/This-is-PL-Interactive-Combined.pdf"
tabula.convert_into(combined_pdf, folder_location +"/table_output.csv", output_format="csv",pages = 18,area=[[275,504,640,900]])
pd.read_csv(folder_location+"/table_output.csv", encoding= "ISO-8859-1")

,Pos,Unnamed: 1,Club,W,D,L,GD,Pts,Total payment
0,1.0,NaN,Manchester City,32.0,4.0,2.0,79.0,100.0,Â£149.4m
1,2.0,NaN,Manchester United,25.0,6.0,7.0,40.0,81.0,Â£149.8m
2,3.0,NaN,Tottenham Hotspur,23.0,8.0,7.0,38.0,77.0,Â£144.4m
3,4.0,NaN,Liverpool,21.0,12.0,5.0,46.0,75.0,Â£145.9m
4,5.0,NaN,Chelsea,21.0,7.0,10.0,24.0,70.0,Â£141.7m
5,6.0,NaN,Arsenal,19.0,6.0,13.0,23.0,63.0,Â£142.0m
6,7.0,NaN,Burnley,14.0,12.0,12.0,-3.0,54.0,Â£119.8m
7,8.0,NaN,Everton,13.0,10.0,15.0,-14.0,49.0,Â£128.0m
8,9.0,NaN,Leicester City,12.0,11.0,15.0,-4.0,47.0,Â£118.2m
9,10.0,NaN,Newcastle United,12.0,8.0,18.0,-8.0,44.0,Â£123.0m




---



---



#5. Cleaning Data using Excel

a. Find and replace (Ctrl+H)\
b. Changing data format (Home tab)\
c. Replace extra spaces using Trim\
d. To find blanks and delete the row containg it, do Find&Select -> Go To Special -> Blanks -> Right click -> Delete row\
e. To remove duplicates do Data -> Remove Duplicates



---



---



#6. Transforming Data in Excel
a. To remove outliers, use Pivot Table, plot frequency chart and check\
b. Split text into multiple columns using Data -> Text to Columns\
c. Extract month from date -> =text([@date],'mmm')\
d. Extract year from date -> =text([@date],'yyyy')\
e. Extract week number from date -> =weeknum([@date],1)




---



---



#7. Data Aggregation
a. Remove columns with empty values/ unnecessary columns\
b. Remove rows with empty values as before\
c. Convert data as a table in Excel to exploit some good features of tables\
d. Identify clusters using Color Scales by
Select Column -> Conditional Formatting -> Color Scales\
e. Create Pivot Table and analyse\
f. Create Sparklines, mark high and low point markers; create Data Bars to show growth/decline



---



---



#8. Profiling with Pandas
Given a data frame, it will generate a HTML report containing information like outliers, correlation, etc.\

Not working on collab, use it on PyCharm

In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
#Load the webpage
r = requests.get("https://www.cricbuzz.com/cricket-match/live-scores")
# Convert to a beautiful soup object

soup = bs(r.content, 'html.parser')
# Print out HTML
contents = soup.find_all('window.onerror')
print(contents)

[]


In [ ]:
url = 'https://drive.google.com/file/d/1KjrSid8AfVggkCX3pmcpRE-OAi8CHVUr/view?usp=sharing'
url2 = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url2, encoding='latin-1')
df.describe()

,UN 2018 population estimates[b],City proper[c] Population,City proper[c] Area (km2),City proper[c] Density (/km2),Metropolitan area[d] Population,Metropolitan area[d] Area (km2),Metropolitan area[d] Density (/km2),Urban area[12] Population,Urban area[12] Area (km2),Urban area[12] Density (/km2)
count,8.100000e+01,7.400000e+01,74.000000,74.000000,4.100000e+01,31.000000,31.000000,8.000000e+01,80.000000,80.00000
mean,1.073674e+07,7.783677e+06,4935.378378,7261.108108,1.349890e+07,9579.709677,3349.225806,1.185228e+07,2324.275000,8086.88750
std,6.276120e+06,5.873206e+06,11766.341357,7196.955453,8.183469e+06,6512.711300,4746.479437,7.303674e+06,2095.803456,6723.32797
min,5.023000e+06,2.364530e+05,22.000000,29.000000,5.156217e+06,620.000000,274.000000,2.280000e+06,238.000000,734.00000
25%,6.115000e+06,2.726647e+06,358.000000,1890.000000,6.641649e+06,4067.500000,774.000000,6.521000e+06,1025.250000,4036.50000
50%,8.245000e+06,7.697000e+06,1307.000000,5163.000000,1.254527e+07,7762.000000,2094.000000,9.143500e+06,1646.000000,5743.00000
75%,1.321500e+07,1.051551e+07,3768.500000,10756.250000,1.930300e+07,14427.500000,3083.500000,1.547950e+07,3064.750000,10012.50000
max,3.740007e+07,3.205416e+07,82403.000000,41399.000000,3.727400e+07,22463.000000,20770.000000,3.910500e+07,12093.000000,36928.00000
